In [3]:
import chromadb
import os

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings

from chromadb.utils import embedding_functions

In [4]:
path_client = "./client"

def get_client():
    return chromadb.PersistentClient(path=path_client)

In [5]:
chroma_client = get_client()

In [6]:
pdf_folder_path = "./raw_files"
documents = []

for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

In [9]:
chroma_client.delete_collection(name="sc_collection")

In [10]:
collection = chroma_client.create_collection("sc_collection")

In [11]:
embedding_function = embedding_functions.DefaultEmbeddingFunction()

In [12]:
chunked_documents[0]

Document(metadata={'producer': 'LibreOffice 25.2.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2025-03-14T13:00:13+01:00', 'source': './raw_files/Cycle 3 - Ateliers Jeunes - Ma place dans le monde numérique.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='Diapo 1 \nPas de message\nDiapo 2 \nBienvenue,\nPendant cet atelier, tu vas pouvoir explorer des sujets importants autour de ton \ncomportement en ligne. Voici trois choses à retenir avant de commencer : \nPremièrement, tu es anonyme.\nTes réponses aux questions posées lors de cet atelier resteront confidentielles. Cela veut \ndire que tu peux répondre librement, sans que personne ne sache ce que tu as dit.\nDeuxièmement, si un sujet te préoccupe ou si tu rencontres un problème, il est important de \nsavoir qu’il y a des personnes prêtes à t’aider :\n○ Le bouton violet te permet d’être contacté par le référent de ton académie.\n○ Le bouton vert te permet de joindre un conseiller du 3

In [36]:
i = 0
for doc in chunked_documents:
    
    embeddings_doc = embedding_function(doc.page_content)
    collection.add(
        documents = doc.page_content,
        embeddings = [embeddings_doc[0].tolist()],
        metadatas = doc.metadata,
        ids=[str(i)]
    )
    print(f'doc {i}/{len(chunked_documents)} added')
    i += 1

KeyboardInterrupt: 

In [34]:
collection.peek()

{'ids': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'embeddings': array([[-0.11551929,  0.0431053 , -0.00964827, ...,  0.02105021,
         -0.05008018,  0.00334117],
        [-0.11551929,  0.0431053 , -0.00964827, ...,  0.02105021,
         -0.05008018,  0.00334117],
        [-0.01828039,  0.07142014, -0.06651127, ...,  0.03860461,
         -0.00139447,  0.03592809],
        ...,
        [-0.04983491,  0.04741006,  0.02075684, ...,  0.00342485,
         -0.01610464, -0.00163354],
        [-0.08815008,  0.0389121 , -0.06267454, ...,  0.02407954,
          0.06344451,  0.02097689],
        [-0.11551929,  0.0431053 , -0.00964827, ...,  0.02105021,
         -0.05008018,  0.00334117]]),
 'documents': ['Diapo 1 \nPas de message\nDiapo 2 \nBienvenue,\nPendant cet atelier, tu vas pouvoir explorer des sujets importants autour de ton \ncomportement en ligne. Voici trois choses à retenir avant de commencer : \nPremièrement, tu es anonyme.\nTes réponses aux questions posées lors de cet 